### Generates Provider Features using NPI Database

In [ ]:
## This notebook reads the in_name df from the database, and writes to the out_name database
in_name= 'manuscript_covariates_0_final'
out_name = 'manuscript_covariates_1_final'

In [ ]:
import sys
import pickle
import requests
import json
sys.path.append( '..' )
import psycopg2
import pandas as pd
import numpy
#import sparse
import config
import sys
import time
import importlib
#
import datetime
import pandas as pd
import numpy as np
import scipy.sparse
import matplotlib.pyplot as plt



In [ ]:
import sparse
import Utils.dbutils as dbutils
import Utils.data_utils as data_utils
import Generators.CohortGenerator as CohortGenerator
import Generators.FeatureGenerator as FeatureGenerator
import config
local_imports = (
    dbutils,
    data_utils,
    CohortGenerator,
    FeatureGenerator,
    config
)
for i in local_imports:
    i = importlib.reload(i)

# database connection
username = config.PG_USERNAME
password = config.PG_PASSWORD
database_name = config.DB_NAME

#config_path = 'postgresql://{username}:{password}@{database_name}'.format(
config_path = 'postgresql://{database_name}'.format(database_name = database_name)

connect_args = {"host": '/var/run/postgresql/'} # connect_args to pass to sqlalchemy create_engine function

# schemas 
# schema_name = 'eol_test_ncjones' # all created tables will be created using this schema
cdm_schema_name = config.OMOP_CDM_SCHEMA # the name of the schema housing your OMOP CDM tables
print(f"cdm schema: {cdm_schema_name}")

    #username = username,
    #password = password,
   # database_name = database_name
#)

# schemas 
schema_name = 'eol_test_eheal' # all created tables will be created using this schema

# caching
reset_schema = False # if true, rebuild all data from scratch

# set up database, reset schemas as needed
db = dbutils.Database(config_path, schema_name, connect_args, cdm_schema_name)
# if reset_schema:
#     db.execute(
#         'drop schema if exists {} cascade'.format(schema_name)
#     )
# db.execute(
#     'create schema if not exists {}'.format(schema_name)
# )

In [ ]:
# set input
username='cdm_6871_21'
password='MLHC2021'
database_name = 'omop_v6'
conn_string = "dbname="+database_name + " host=/var/run/postgresql"
conn=psycopg2.connect(conn_string)

sql_cmd = 'select * from cdm_6871_21.{}'.format(in_name)
covariate_df =pd.read_sql(sql_cmd, conn)

 

## Add provider info

In [ ]:
%%time


# visit provider info
sql_cmd = '''
SELECT provider.provider_id, specialty_concept_id, concept_name, npi
FROM (SELECT * FROM cdm_6871_21.provider
        WHERE provider_id IN
            (SELECT visit_provider_id FROM cdm_6871_21.cohort_manuscript_updated_treatments_ncj)) as provider
JOIN (SELECT concept_id, concept_name FROM cdm_6871_21.concept) as concept
ON provider.specialty_concept_id=concept.concept_id
GROUP BY provider.provider_id, provider.specialty_concept_id, concept.concept_name, provider.npi
'''
provider_visit = pd.read_sql(sql_cmd, conn)
provider_visit.head()
print(len(provider_visit))


In [ ]:
%%time 
# durg provider infor
sql_cmd = '''
SELECT provider.provider_id, specialty_concept_id, concept_name, npi
FROM (SELECT * FROM cdm_6871_21.provider
        WHERE provider_id IN
            (SELECT provider_id FROM cdm_6871_21.cohort_manuscript_updated_treatments_ncj)) as provider
JOIN (SELECT concept_id, concept_name FROM cdm_6871_21.concept) as concept
ON provider.specialty_concept_id=concept.concept_id
GROUP BY provider.provider_id, provider.specialty_concept_id, concept.concept_name, provider.npi
'''
provider = pd.read_sql(sql_cmd, conn)
provider.head()
print(len(provider))

In [ ]:
provider_visit = provider_visit.rename(columns = {'specialty_concept_id': 'visit_specialty_concept_id','provider_id':'visit_provider_id','npi':'visit_npi','concept_name':'visit_concept_name'})
provider = provider.rename(columns = {'npi':'drug_npi'})



In [ ]:
#vtemp_visit = v12.join(provider_visit[['visit_provider_id', 'visit_specialty_concept_id', 'visit_concept_name','visit_npi']].set_index('visit_provider_id'), on='visit_provider_id')
vtemp_visit = covariate_df.join(provider_visit[['visit_provider_id', 'visit_specialty_concept_id', 'visit_concept_name','visit_npi']].set_index('visit_provider_id'), on='visit_provider_id')
vtemp_visit['visit_specialty_concept_id'].fillna(0, inplace = True)

vtemp = vtemp_visit.join(provider[['provider_id', 'specialty_concept_id', 'concept_name','drug_npi']].set_index('provider_id'), on='provider_id')
vtemp['specialty_concept_id'].fillna(0, inplace = True)


In [ ]:
temp = vtemp[['npi','drug_npi','visit_npi','condition_occurrence_id','person_id','visit_provider_id','provider_id', 'visit_specialty_concept_id', 'specialty_concept_id', 'visit_concept_name','concept_name']]



In [ ]:
temp1 = temp.assign(provider_specialty_available = False)
temp1.provider_specialty_available = (~(temp1.concept_name=='No matching concept')&~(temp1.provider_id==-1))

temp1 = temp1.assign(visit_provider_specialty_available = False)
temp1.visit_provider_specialty_available = (~(temp1.visit_concept_name=='No matching concept')&~(temp1.visit_provider_id==-1))
provider_table = temp1.copy()


## Query NPI database and Pickle

In [ ]:
%%time
error_index = []
error_condition = []
not_found = []

provider_visit_dict = {}
count = 0
for n in provider_table.visit_npi.unique():
    count+=1
    if count%200==0:
        print(count/provider_table.visit_npi.nunique())
    try:
        r = requests.get(f'https://npiregistry.cms.hhs.gov/api/?version=2.1&number={n}&pretty=on')
        results_text =json.loads(r.text)
        try: 
            provider_visit_dict[n] = results_text["results"][0]["taxonomies"]
        except:
            not_found.append(n)          
    except:
        error_condition.append(n)
        error_index.append(n)
print (f'Number of not found: {len(not_found)}')



In [ ]:
import pickle
pickle.dump(provider_visit_dict, open('provider_visit_dict.p','wb'))
pickle.dump(not_found, open('provider_visit_npis_not_found.p','wb'))

In [ ]:
%%time
# Load Drug NPI information
error_index_drug = []
error_condition_drug = []
not_found_drug = []

provider_drug_dict = {}
count = 0
for n in provider_table.drug_npi.unique():
    count+=1
    if count%600==0:
        print(count/provider_table.drug_npi.nunique())
    try:
        r = requests.get(f'https://npiregistry.cms.hhs.gov/api/?version=2.1&number={n}&pretty=on')
        results_text =json.loads( r.text)
        try: 
            provider_drug_dict[n] = results_text["results"][0]["taxonomies"]
        except:
            not_found_drug.append(n)          
    except:
        error_condition_drug.append(n)
        error_index_drug.append(n)
import pickle
pickle.dump(provider_drug_dict, open('provider_drug_dict.p','wb'))
pickle.dump(not_found_drug, open('provider_drug_npis_not_found.p','wb'))


## Load pickle and add information to covariate table

In [ ]:
provider_visit_dict =pickle.load(open('provider_visit_dict.p','rb'))
provider_drug_dict =pickle.load(open('provider_drug_dict.p','rb'))

In [ ]:
%%time
# Drug
provider_table = provider_table.assign(visit_npi_desc = '')
provider_table = provider_table.assign(drug_npi_desc = '')
provider_table = provider_table.assign(npi_same = False)

error_index = []
error_condition = []
not_found = []
no_primary_taxonomy_found_drug = []
no_primary_taxonomy_found_visit = []
flag = False
count = 0
multiple = []
for idx,row in provider_table.iterrows():
    n = row.drug_npi
    if str(n) !='nan':
        try:
            taxonomies =provider_drug_dict[n]
            if len(taxonomies)==1:  
                provider_table.loc[idx,'drug_npi_desc'] = taxonomies[0]["desc"]
            else:
                num_tax = 0
                for tax in taxonomies:
                    
                    multiple.append(n)
                    if tax['primary']==True:
                        provider_table.loc[idx,'drug_npi_desc'] = tax["desc"]
                        num_tax +=1
                        flag = True
                if num_tax>1:
                    print(n)
                if flag ==False:
                    no_primary_taxonomy_found_drug.append(n)
                flag = False
        except:
            provider_table.loc[idx,'drug_npi_desc'] = None
    n = row.visit_npi
    if str(n) != 'nan' and n != None:
        try:
            taxonomies =provider_visit_dict[n]
            
            if len(taxonomies)==1:  
                provider_table.loc[idx,'visit_npi_desc'] = taxonomies[0]["desc"]
            else:
                for tax in taxonomies:
                    if tax['primary']==True:
                        provider_table.loc[idx,'visit_npi_desc'] = tax["desc"]
                        flag = True
                        
                if flag ==False:
                    no_primary_taxonomy_found_visit.append(n)
                flag = False
            if row.drug_npi ==row.visit_npi:
                provider_table.loc[idx,'npi_same'] = True    
            count +=1
        except:
            provider_table.loc[idx,'visit_npi_desc'] = None

        if count%5000 ==0:
            print(count/len(provider_table))


In [ ]:
covariate_df_new = covariate_df.join(provider_table[['condition_occurrence_id', 'drug_npi_desc']].set_index('condition_occurrence_id'), on='condition_occurrence_id')

### One hot encoding


In [ ]:
covariate_df_new = covariate_df_new.assign(provider_specialty_available = False)
covariate_df_new.provider_specialty_available = (~(covariate_df_new['drug_npi_desc']==''))

In [ ]:
%%time
covariate_df_new1 = covariate_df_new1.assign(drug_npi_desc_grouped = '')
none_count = 0
i=0
for idx, row in covariate_df_new1.iterrows():
    if row.drug_npi_desc == None:
        none_count+=1
    else:
        desc = row.drug_npi_desc.lower()
        if 'nurse' in desc or 'physician assistant' in desc:
            covariate_df_new1.loc[idx,'drug_npi_desc_grouped'] = 'advanced_specialist_group'
        elif ('family' in desc and not ('planning' in desc or 'marriage' in desc) ) or  ('general practice' in desc and not 'dentist' in desc):
            covariate_df_new1.loc[idx,'drug_npi_desc_grouped'] = 'family_medicine_group'
        elif 'internal medicine' == desc or 'internal medicine geriatric medicine' == desc:
            covariate_df_new1.loc[idx,'drug_npi_desc_grouped'] = 'internal_medicine_group'
        elif 'emergency' in desc or 'acute' in desc or 'hospital' in desc:
            covariate_df_new1.loc[idx,'drug_npi_desc_grouped'] = 'emergency/acute_group'
        elif 'urology' in desc and not('neurology' in desc):
            covariate_df_new1.loc[idx,'drug_npi_desc_grouped'] = 'urology_group'
        elif 'gynecology' in desc or 'OBG' in desc or 'midwife' in desc:
            covariate_df_new1.loc[idx,'drug_npi_desc_grouped'] = 'OBGYN_group'
        elif len(desc)>0 :
            covariate_df_new1.loc[idx,'drug_npi_desc_grouped'] = 'other_group'
    if i %3000 ==0:
        print(i/len(covariate_df_new1))
        
    i = i+1

In [ ]:
covariate_df_new1.provider_specialty_available =   (covariate_df_new1.drug_npi_desc_grouped!='')

In [ ]:
covariate_df_new1[covariate_df_new1['drug_npi_desc_grouped']=='family_medicine_group']

In [ ]:
specialties = pd.DataFrame(covariate_df_new1.drug_npi_desc_grouped.value_counts()).index
specialties = specialties.drop('')
specialties_list =[]
for s in list(specialties):
    specialties_list.append('specialty_'+str(s))


In [ ]:
temp = covariate_df_new1[['drug_npi_desc_grouped']]
onehot = pd.get_dummies(temp,prefix = 'specialty')

In [ ]:
covariate_df_new2 = covariate_df_new1.merge(onehot.loc[:,specialties_list],left_index = True,right_index = True)

In [ ]:
covariate_df_new2 = covariate_df_new2.drop(columns = ['drug_npi_desc'])

In [ ]:
covariate_df_new2 = covariate_df_new2.drop(columns = ['drug_npi_desc_grouped'])

In [ ]:
covariate_df_new2 =covariate_df_new2.drop(columns = 'level_0')

In [ ]:
covariate_df_new2.to_sql("{}".format(out_name),con=db.engine, if_exists="replace", schema="cdm_6871_21")


In [ ]:
db.execute("GRANT SELECT ON TABLE cdm_6871_21.{} TO cdm_6871_21;".format(out_name))



# Stop

## Analysis 

In [ ]:
# provider_table[provider_table.drug_npi.isna()].visit_npi_desc.value_counts().head(30)

In [ ]:
# provider_table[provider_table.drug_npi.isna()]

In [ ]:
# tmp = provider_table.assign(drug_npi_desc_grouped = '')

# i=0
# for idx, row in tmp.iterrows():
#     desc = row.drug_npi_desc.lower()
#     if 'nurse' in desc or 'physician assistant' in desc:
#         tmp.loc[idx,'drug_npi_desc_grouped'] = 'advanced_specialist_group'
#     elif 'family' in desc and not ('planning' in desc or 'marriage' in desc):
#         tmp.loc[idx,'drug_npi_desc_grouped'] = 'family_medicine_group'
#     elif 'internal' in desc or 'general practice' in desc:
#         tmp.loc[idx,'drug_npi_desc_grouped'] = 'internal_medicine_group'
#     elif 'emergency' in desc or 'acute' in desc or 'hospital' in desc:
#         tmp.loc[idx,'drug_npi_desc_grouped'] = 'emergency/acute_group'
#     elif 'urology' in desc and not('neurology' in desc):
#         tmp.loc[idx,'drug_npi_desc_grouped'] = 'urology_group'
#     elif 'gynecology' in desc or 'OBG' in desc or 'midwife' in desc:
#         tmp.loc[idx,'drug_npi_desc_grouped'] = 'OBGYN_group'
#     elif len(desc)>0 :
#         tmp.loc[idx,'drug_npi_desc_grouped'] = 'other_group'
#     if i %3000 ==0:
#         print(i)
        
#     i = i+1

In [ ]:
# tmp = tmp.assign(visit_npi_desc_grouped = '')

# i=0
# for idx, row in tmp.iterrows():
#     desc = row.visit_npi_desc.lower()
#     if 'nurse' in desc or 'physician assistant' in desc:
#         tmp.loc[idx,'visit_npi_desc_grouped'] = 'advanced_specialist_group'
#     elif 'family' in desc and not ('planning' in desc or 'marriage' in desc):
#         tmp.loc[idx,'visit_npi_desc_grouped'] = 'family_medicine_group'
#     elif 'internal' in desc or 'general practice' in desc:
#         tmp.loc[idx,'visit_npi_desc_grouped'] = 'internal_medicine_group'
#     elif 'emergency' in desc or 'acute' in desc or 'hospital' in desc:
#         tmp.loc[idx,'visit_npi_desc_grouped'] = 'emergency/acute_group'
#     elif 'urology' in desc and not('neurology' in desc):
#         tmp.loc[idx,'visit_npi_desc_grouped'] = 'urology_group'
#     elif 'gynecology' in desc or 'OBG' in desc or 'midwife' in desc:
#         tmp.loc[idx,'visit_npi_desc_grouped'] = 'OBGYN_group'
#     elif len(desc)>0 :
#         tmp.loc[idx,'visit_npi_desc_grouped'] = 'other_group'
#     if i %3000 ==0:
#         print(i)
        
#     i = i+1

In [ ]:
# tmp2 = tmp[tmp.drug_npi_desc_grouped != '']

In [ ]:
# tmp3 =  tmp2[tmp2.visit_npi_desc_grouped != '']

In [ ]:
# tmp3

In [ ]:
# df = tmp3[['drug_npi_desc_grouped','visit_npi_desc_grouped']]

In [ ]:
# df

In [ ]:
# final = df.groupby(['drug_npi_desc_grouped','visit_npi_desc_grouped']).size()

In [ ]:
# temp = pd.DataFrame(final)

In [ ]:
# temp

In [ ]:
# temp.sort_values(by = (0),ascending = False).head(50)